[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JialinMao/private_CNN/blob/master/vit.ipynb)


In [1]:
!pip install opacus==1.1.1
!pip install timm

In [3]:
!git clone https://github.com/JialinMao/private_CNN.git

Cloning into 'private_CNN'...
remote: Enumerating objects: 1625, done.
remote: Counting objects: 100% (1625/1625), done.
remote: Compressing objects: 100% (776/776), done.
remote: Total 1625 (delta 834), reused 1622 (delta 831), pack-reused 0
Receiving objects: 100% (1625/1625), 28.55 MiB | 22.42 MiB/s, done.
Resolving deltas: 100% (834/834), done.


In [6]:
cd private_CNN/

/content/private_CNN


In [11]:
import sys
sys.path.insert(0, '/content/private_CNN')
import private_CNN

In [10]:
import timm
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

from opacus.validators import ModuleValidator
from opacus.accountants.utils import get_noise_multiplier

In [12]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Cifar10 torchvision models

## Arguments

In [14]:
lr = 0.1
epochs = 20
bs = 128
eps = 2
grad_norm = 0.1
mode = 'ghost-mixed'
model = 'crossvit_18_240'
mini_batch_size =  50
pretrained = True
cifar_data = 'CIFAR100'

## Loading Data

In [15]:
print('==> Preparing data..')

transform_train = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
])
transform_test = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
])

trainset = torchvision.datasets.CIFAR100(
    root='../../data', train=True, download=True, transform=transform_train)
testset = torchvision.datasets.CIFAR100(
    root='../../data', train=False, download=True, transform=transform_test)

trainloader = torch.utils.data.DataLoader(
        trainset, batch_size=mini_batch_size, shuffle=True, num_workers=2)

testloader = torch.utils.data.DataLoader(
        testset, batch_size=100, shuffle=False, num_workers=2)



==> Preparing data..


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ../../data/cifar-10-python.tar.gz to ../../data
Files already downloaded and verified


## Building Model

In [16]:
print('==> Building model..', model, '  mode ', mode)
NUM_CLASSES=10 if cifar_data=='CIFAR10' else 100

net = timm.create_model(model,pretrained=pretrained,num_classes=NUM_CLASSES)
net = ModuleValidator.fix(net)

if 'convit' in model:
    for name,param in net.named_parameters():
        if 'attn.gating_param' in name:
            param.requires_grad=False
if 'beit' in model:
    for name,param in net.named_parameters():
        if 'gamma_' in name or 'relative_position_bias_table' in name or 'attn.qkv.weight' in name or 'attn.q_bias' in name or 'attn.v_bias' in name:
            requires_grad=False


for name,param in net.named_parameters():
    if 'cls_token' in name or 'pos_embed' in name:
        param.requires_grad=False

if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

print('number of parameters: ', sum([p.numel() for p in net.parameters()]))


==> Building model..
number of parameters:  11173962


## Privacy Engine

In [ ]:
if "ghost" in mode:
    criterion = nn.CrossEntropyLoss(reduction="none")
else:
    criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(net.parameters(), lr=lr)

n_acc_steps = bs // mini_batch_size

if 'ghost' in mode:
    sigma = get_noise_multiplier(
            target_epsilon = eps,
            target_delta = 1e-5,
            sample_rate = bs/len(trainset),
            epochs = epochs,
            accountant = "gdp"
        )
    privacy_engine = private_CNN.PrivacyEngine(
        net,
        batch_size=bs,
        sample_size=len(trainloader.dataset),
        noise_multiplier=sigma,
        epochs=epochs,
        max_grad_norm=grad_norm,
        ghost_clipping=True,
        mixed='mixed' in mode
    )
    privacy_engine.attach(optimizer)



## Trainining and Testing

In [ ]:
# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(tqdm(trainloader)):
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = net(inputs)
        loss = criterion(outputs, targets)

        if mode=='non-private':
            loss.backward()
            if ((batch_idx + 1) % n_acc_steps == 0) or ((batch_idx + 1) == len(trainloader)):
                optimizer.step()
                optimizer.zero_grad()
        else:
            if ((batch_idx + 1) % n_acc_steps == 0) or ((batch_idx + 1) == len(trainloader)):
                optimizer.step(loss=loss)
                optimizer.zero_grad()
            else:
                optimizer.virtual_step(loss=loss)
        train_loss += loss.mean().item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    print(epoch, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                        % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))

def test(epoch):
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(tqdm(testloader)):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            loss = loss.mean()
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

        print(epoch, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                            % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))


In [ ]:
for epoch in range(epochs):
    train(epoch)
    test(epoch)